#### Challenge

In [ ]:
from utils import listener
from sage.all import *

FLAG = b"crypto{????????????????????????}"

# dimension
n = 64
# plaintext modulus
p = 257
# ciphertext modulus
q = 1048583

V = VectorSpace(GF(q), n)
S = V.random_element()

def encrypt(m):
    A = V.random_element()
    e = randint(-1, 1)
    b = A * S + m + p * e
    return A, b

class Challenge:
    def __init__(self):
        self.before_input = "Would you like to encrypt your own message, or see an encryption of a character in the flag?\n"

    def challenge(self, your_input):
        if 'option' not in your_input:
            return {'error': 'You must specify an option'}

        if your_input['option'] == 'get_flag':
            if "index" not in your_input:
                return {"error": "You must provide an index"}
                self.exit = True

            index = int(your_input["index"])
            if index < 0 or index >= len(FLAG) :
                return {"error": f"index must be between 0 and {len(FLAG) - 1}"}
                self.exit = True

            A, b = encrypt(FLAG[index])
            return {"A": str(list(A)), "b": str(int(b))}

        elif your_input['option'] == 'encrypt':
            if "message" not in your_input:
                return {"error": "You must provide a message"}
                self.exit = True

            message = int(your_input["message"])
            if message < 0 or message >= p:
                return {"error": f"message must be between 0 and {p - 1}"}
                self.exit = True

            A, b = encrypt(message)
            return {"A": str(list(A)), "b": str(int(b))}

        return {'error': 'Unknown action'}


import builtins; builtins.Challenge = Challenge # hack to enable challenge to be run locally, see https://cryptohack.org/faq/#listener
listener.start_server(port=13413)


In [30]:
from sage.all import *
from pwn import *
import json
from Crypto.Util.number import *

# dimension
n = 64
m = 128
# plaintext modulus
p = 257
# ciphertext modulus
q = 1048583

host = 'socket.cryptohack.org'
port = 13413
r = remote(host, port)

rows = []
lhs = []

r.recvuntil(b'encryption of a character in the flag?\n')

for i in tqdm(range(m)):
    payload = {
        "option": "encrypt",
        "message": "0"
    }

    payload = json.dumps(payload).encode()
    r.sendline(payload)

    response = json.loads(r.recvline().strip().decode())
    A = eval(response["A"])
    b = int(response["b"])

    for x in range(n):
        A[x] = (A[x] * pow(p, -1, q)) % q
    b = (b * pow(p, -1, q)) % q

    rows.append(A)
    lhs.append(b)

A = Matrix(ZZ, rows)
b = vector(ZZ, lhs)

M = Matrix(ZZ,  m + n + 1, m)
for i in range(m):
    M[i, i] = q 

for i in range(m):
    for j in range(n):
        M[m + j, i] = A[i, j]

for i in range(m):
    M[m + n, i] = -b[i]

# print(M)
M = M.LLL()
# print(M)

for i in range(m + n + 1):
    found = False
    for j in range(m):
        if M[i, j] != 0:
            found = True 

    if found:
        e = M[i]
        break 

f = []
for i in range(n):
    f.append(-1 * e[i])

f = vector(GF(q), f)

rows = []
for i in range(n):
    rows.append(b[i])

b_ = vector(GF(q), rows) 

rows = []
for i in range(n):
    rows.append(A[i])

A_ = Matrix(GF(q), rows)

x = b_ - f 
s = A_ ** (-1) * x 
assert b == A * s - e

flag = '' 
i = 0

while True:
    payload = {
        "option": "get_flag",
        "index": str(i)
    }
    payload = json.dumps(payload).encode()
    r.sendline(payload)

    response = json.loads(r.recvline().strip().decode())
    A = eval(response["A"])
    b = int(response["b"])

    A = vector(GF(q), A)
    x = (b - A * s) % q

    if x > q - 2 * 257:
        x = int((x + 257) % q) 
        flag += chr(x)
    elif x < 150:
        flag += chr(int(x))
    else:
        x = int(x - 257)
        flag += chr(x)
    
    print(flag)
    i += 1


[x] Opening connection to socket.cryptohack.org on port 13413


[x] Opening connection to socket.cryptohack.org on port 13413: Trying 134.122.111.232
[+] Opening connection to socket.cryptohack.org on port 13413: Done


100%|██████████| 128/128 [00:19<00:00,  6.59it/s]


c
cr
cry
cryp
crypt
crypto
crypto{
crypto{L
crypto{LL
crypto{LLL
crypto{LLL_
crypto{LLL_i
crypto{LLL_is
crypto{LLL_is_
crypto{LLL_is_a
crypto{LLL_is_al
crypto{LLL_is_als
crypto{LLL_is_also
crypto{LLL_is_also_
crypto{LLL_is_also_v
crypto{LLL_is_also_ve
crypto{LLL_is_also_ver
crypto{LLL_is_also_very
crypto{LLL_is_also_very_
crypto{LLL_is_also_very_u
crypto{LLL_is_also_very_us
crypto{LLL_is_also_very_use
crypto{LLL_is_also_very_usef
crypto{LLL_is_also_very_usefu
crypto{LLL_is_also_very_useful
crypto{LLL_is_also_very_useful!
crypto{LLL_is_also_very_useful!}


KeyError: 'A'